In [117]:
using ModelingToolkit,DifferentialEquations,Plots
using ModelingToolkit: t_nounits as t, D_nounits as D 

In [106]:
abstract type Laser end
@kwdef struct Gaussian <: Laser 
    FWHM::Real
    Offset::Real
    Power::Real
    hv::Real
    spatial::Array{Float64}
    R::Real
end
@kwdef struct Rectangular <: Laser    
    FWHM::Real
    Offset::Real
    Power::Real
    hv::Real
    spatial::Array{Float64}
    R::Real
end

Rectangular

In [107]:
function (::Gaussian)(;name)
    @parameters FWHM  Offset
    @variables tp(t)
    
    eqs = [tp ~ sqrt(4*log(2)/pi)/FWHM*exp(-4*log(2)*(t-(2*FWHM)-Offset)^2/FWHM^2)]
    
    ODESystem(eqs,t;name)
end

In [108]:
function tester(;name)
    @variables S(t) T(t)
    eqs = [D(T) ~ S]

    ODESystem(eqs,t;name)
end

tester (generic function with 1 method)

In [109]:
lp=Gaussian(FWHM=100,Offset=300,Power=62,hv=1.55,spatial=[0],R=0.0)
@named gauss_laser=lp()
@named testconnector=tester()

Model testconnector with 1 equations
Unknowns (2):
  T(t)
  S(t)
Parameters (0):

In [110]:
connections = [testconnector.S ~ gauss_laser.tp]
connected = compose(ODESystem(connections,t,name = :connected),gauss_laser,testconnector)

Model connected with 3 equations
Unknowns (3):
  testconnector₊S(t)
  gauss_laser₊tp(t)
  testconnector₊T(t)
Parameters (2):
  gauss_laser₊FWHM
  gauss_laser₊Offset

In [111]:
connected_simp=structural_simplify(connected)

Model connected with 1 equations
Unknowns (1):
  testconnector₊T(t)
Parameters (2):
  gauss_laser₊FWHM
  gauss_laser₊Offset
Incidence matrix:1×2 SparseArrays.SparseMatrixCSC{Num, Int64} with 1 stored entry:
 ⋅  ×

In [114]:
u0=[gauss_laser.tp => 0.0, 
   testconnector.T => 0.0]

2-element Vector{Pair{Num, Float64}}:
  gauss_laser₊tp(t) => 0.0
 testconnector₊T(t) => 0.0

In [115]:
p=[gauss_laser.FWHM => lp.FWHM,
   gauss_laser.Offset => lp.Offset]

2-element Vector{Pair{Num, Int64}}:
   gauss_laser₊FWHM => 100
 gauss_laser₊Offset => 300

In [118]:
prob=ODEProblem(connected_simp,u0,(0.0,500.0),p)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 500.0)
u0: 1-element Vector{Float64}:
 0.0

In [119]:
plot(solve(prob))

In [82]:
function test_eq(;name)
    @parameters a b 
    @variables y(t)

    eqs =  [y ~ a*t + b]

    ODESystem(eqs,t;name)
end

test_eq (generic function with 1 method)

In [84]:
@named testeq=test_eq()

Model testeq with 1 equations
Unknowns (1):
  y(t)
Parameters (2):
  a
  b

In [85]:
testsimp=structural_simplify(testeq)

Model testeq with 0 equations
Unknowns (0):
Parameters (2):
  a
  b
Incidence matrix:0×0 SparseArrays.SparseMatrixCSC{Num, Int64} with 0 stored entries